# Classificação de tópicos por embedding

Importe as bibliotecas necessárias

In [1]:
# Importing libraries
import regex as re
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from langdetect import detect
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import nltk
nltk.download('punkt')
import pandas as pd


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lavin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Separação dos texto por frases

In [2]:
'''
# Load the spacy model
nlp = spacy.load('en_core_web_sm')

# Load data
data = pd.read_csv('all_ECB_speeches.csv', sep='|')

# Function to detect language of a given text
def detect_language(text):
    try:
        lang = detect(text)
    except:
        lang = "Unknown"
    return lang

# Apply the language detection function to each text in your DataFrame
data['language'] = data['contents'].apply(detect_language)

# Filter out the texts that are not in English
data = data[data['language'] == 'en']

# Drop null values
data = data.dropna(subset=['speakers', 'contents'])

# Reset index
data = data.reset_index(drop=True)

# Remove introduction
for row in range(data.shape[0]):
    try:
        speech = re.split(" \d+ (January|February|March|April|May|June|July|August|September|October|November|December) \d{4} ", data.loc[row, "contents"])
        data.loc[row, "contents"] = speech[-1]
    except:
        pass

# Split into sentences using spacy
def split_into_sentences(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences

sentences_list = []
for row in range(data.shape[0]):
    text = data.loc[row, "contents"]
    sentences = split_into_sentences(text)
    sentences_list.append(sentences)
data['sentences'] = sentences_list

# Explode the sentences column to have one sentence per row
sentences_df = data.explode('sentences').reset_index(drop=True)

sentences_df.to_csv('sentences_df.csv', index=False)
'''

'\n# Load the spacy model\nnlp = spacy.load(\'en_core_web_sm\')\n\n# Load data\ndata = pd.read_csv(\'all_ECB_speeches.csv\', sep=\'|\')\n\n# Function to detect language of a given text\ndef detect_language(text):\n    try:\n        lang = detect(text)\n    except:\n        lang = "Unknown"\n    return lang\n\n# Apply the language detection function to each text in your DataFrame\ndata[\'language\'] = data[\'contents\'].apply(detect_language)\n\n# Filter out the texts that are not in English\ndata = data[data[\'language\'] == \'en\']\n\n# Drop null values\ndata = data.dropna(subset=[\'speakers\', \'contents\'])\n\n# Reset index\ndata = data.reset_index(drop=True)\n\n# Remove introduction\nfor row in range(data.shape[0]):\n    try:\n        speech = re.split(" \\d+ (January|February|March|April|May|June|July|August|September|October|November|December) \\d{4} ", data.loc[row, "contents"])\n        data.loc[row, "contents"] = speech[-1]\n    except:\n        pass\n\n# Split into sentences 

Lê o dataframe das sentenças

In [2]:
sentences_df = pd.read_csv('sentences_df.csv')

Lê as predições feitas manualmente e obtem mais frases para classificar

In [ ]:
my_predictions = pd.read_excel('my_predictions.xlsx')
my_predictions_dict = my_predictions.set_index('sentence').to_dict()['prediction']

Coloque as frases classificadas no dataframe de sentenças

In [ ]:
# Filter the sentences and assign them to the topics
sentences_df['topic'] = '-'
for topic, indexes in my_predictions_dict.items():
    print(f"Topic: {topic}")
    print(f"indexes: {indexes}")
    sentences_df.loc[indexes, 'topic'] = topic


Topic: atividadeeconômica
indexes: [ 10279 205159  55063 106553 172883 128330  83286 206581 140232 107881]
Topic: nenhum
indexes: [  3970 115698  65031  97626  45692  43918 257255  57671  47243  69085]
Topic: inflação
indexes: [269204 193650   2953  15728  20422 138777]
Topic: políticamonetária
indexes: [ 39346 188855 157539  72963 236567 147935 258420 179563 177556 256199]


### Treine o modelo

In [ ]:
df_labeled = sentences_df[sentences_df['topic'] != '-']


Crie um embedding para as sentenças classificadas

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Carregar o tokenizer e o modelo
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def embed_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    print('tokenizado')
    with torch.no_grad():
        outputs = model(**inputs)

    print('embeddings')
    # Utilize a média dos embeddings da última camada
    embeddings = outputs.last_hidden_state.mean(dim=1)
    
    return embeddings.squeeze().numpy()

# Gerar embeddings para todas as sentenças
df_labeled['embeddings'] = df_labeled['sentences'].apply(embed_sentence)


tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings


C:\Users\Lavin\AppData\Local\Temp\ipykernel_22972\3404844447.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labeled['embeddings'] = df_labeled['sentences'].apply(embed_sentence)


Separe em treino e teste 

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir os dados em conjuntos de treino e teste
X = list(df_labeled['embeddings'])
y = df_labeled['topic']
# Separe 2 de 0.8 para treino e 0.2 para teste em cada tópico	
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)



Treine o classificador e avalie o conjunto de teste

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Treinar o classificador
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Avaliar o classificador
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))


                    precision    recall  f1-score   support

atividadeeconômica       1.00      0.50      0.67         2
          inflação       1.00      0.50      0.67         2
            nenhum       0.50      1.00      0.67         2
 políticamonetária       0.50      0.50      0.50         2

          accuracy                           0.62         8
         macro avg       0.75      0.62      0.62         8
      weighted avg       0.75      0.62      0.62         8



Classifique as sentenças não rotuladas

In [ ]:
# Filtrar sentenças não rotuladas e gerar embeddings para um intervalo de sentenças na Christine Lagarde 
mask = (sentences_df['topic'] == '-') & (sentences_df['speakers'] == 'Christine Lagarde')
df_unlabeled = sentences_df[mask]
# Gerar embeddings para sentenças não rotuladas
df_unlabeled['embeddings'] = df_unlabeled['sentences'].apply(embed_sentence)
print("Embeddings gerados para sentenças não rotuladas")
# Prever rótulos para sentenças não rotuladas
X_unlabeled = list(df_unlabeled['embeddings'])
y_unlabeled = classifier.predict(X_unlabeled)
df_unlabeled['topic'] = y_unlabeled
print("Rótulos previstos para sentenças não rotuladas")
# Ver as predições
print(df_unlabeled[['sentences', 'topic']])


tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado
embeddings
tokenizado

C:\Users\Lavin\AppData\Local\Temp\ipykernel_22972\3315927256.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unlabeled['embeddings'] = df_unlabeled['sentences'].apply(embed_sentence)
C:\Users\Lavin\AppData\Local\Temp\ipykernel_22972\3315927256.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unlabeled['topic'] = y_unlabeled


Filtre os dados da Christine Lagarde  e classifique-os

In [ ]:
# Junte os dados rotulados e não rotulados da Christine Lagarde
# Filtrar sentenças rotuladas da Christine Lagarde
mask = df_labeled['speakers'] == 'Christine Lagarde'
df_lagarde = df_labeled[mask]
# Juntar dados rotulados e não rotulados
df_lagarde = pd.concat([df_lagarde, df_unlabeled])

In [ ]:
df_lagarde

,date,speakers,title,subtitle,contents,language,sentences,topic,embeddings
3970,2023-08-25,Christine Lagarde,Policymaking in an age of shifts and breaks,"Speech by Christine Lagarde, President of the ...","Over the past three years, people around the w...",en,"Since our policies operate with lags, we canno...",nenhum,"[0.11813372, 0.17532216, 0.19642851, 0.1106148..."
10279,2022-11-04,Christine Lagarde,Monetary policy in a high inflation environmen...,"Lecture by Christine Lagarde, President of the...","Inflation in the euro area is far too high, re...",en,"Here in Estonia, wage growth is currently runn...",atividadeeconômica,"[-0.20283772, -0.17970902, 0.09091441, 0.06936..."
15728,2022-03-17,Christine Lagarde,Monetary policy in an uncertain world,"Speech by Christine Lagarde, President of the ...",The Russian invasion of Ukraine has cast a sha...,en,This will allow us to test whether the converg...,inflação,"[-0.109350234, -0.066362284, 0.21218097, 0.176..."
0,2024-02-26,Christine Lagarde,European Parliament plenary debate on the ECB ...,"Speech by Christine Lagarde, President of the ...",As we approach the end of this legislative ter...,en,As we approach the end of this legislative ter...,políticamonetária,"[-0.14305893, 0.050547387, 0.4017304, -0.04295..."
1,2024-02-26,Christine Lagarde,European Parliament plenary debate on the ECB ...,"Speech by Christine Lagarde, President of the ...",As we approach the end of this legislative ter...,en,"Building on this collaborative spirit, today’s...",nenhum,"[-0.10264022, 0.17455143, 0.17161143, -0.04376..."
...,...,...,...,...,...,...,...,...,...
36600,2019-11-04,Christine Lagarde,Laudatory speech,"Laudatory speech by Christine Lagarde, Preside...",The English philosopher John Stuart Mill once ...,en,[2]According to the latest Eurobarometer surve...,atividadeeconômica,"[-0.16880375, -0.33953464, 0.6681823, 0.153615..."
36601,2019-11-04,Christine Lagarde,Laudatory speech,"Laudatory speech by Christine Lagarde, Preside...",The English philosopher John Stuart Mill once ...,en,"[3]Schäuble-Lamers Paper, 1 September 1994.",atividadeeconômica,"[-0.35799778, -0.0641206, 0.088410124, -0.2575..."
36602,2019-11-04,Christine Lagarde,Laudatory speech,"Laudatory speech by Christine Lagarde, Preside...",The English philosopher John Stuart Mill once ...,en,"[4]Interview with the Financial Times, 6 Octob...",nenhum,"[-0.16104487, -0.34534147, 0.31121176, -0.2704..."
36603,2019-11-04,Christine Lagarde,Laudatory speech,"Laudatory speech by Christine Lagarde, Preside...",The English philosopher John Stuart Mill once ...,en,"[5]Interview with the Financial Times, 22 Marc...",nenhum,"[-0.13673016, -0.42101058, 0.41494745, -0.2264..."
